You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [26]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [1]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os
from dotenv import load_dotenv


openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [6]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [10]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [11]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [14]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=1
)

Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Title: Unveiling the Future: Exploring the Latest Trends and Players in Artificial Intelligence

Outline:
I. Introduction
- Brief overview of Artificial Intelligence (AI) and its impact on various industries.
- Thesis statement: Delving into the latest trends, top players, and noteworthy news in the ever-evolving field of AI.

II. Latest Trends in Artificial Intelligence
A. Machine Learning Advances
- Discuss the recent advancements in machine learning algorithms and their applications.
B. Natural Language Processing (NLP) Developments
- Explore the breakthroughs in NLP technologies, inclu

- Display the results of your execution as markdown in the notebook.

In [ ]:
result

CrewOutput(raw="# Unveiling the Future: Exploring the Latest Trends and Players in Artificial Intelligence\n\n## Introduction\n\nArtificial Intelligence (AI) has rapidly transformed from a futuristic concept to a ubiquitous presence in today's world, impacting various industries and reshaping the way we interact with technology. This blog post aims to delve into the latest trends, top players, and noteworthy news in the ever-evolving field of AI, offering insights into the advancements driving innovation and the key contributors shaping the industry landscape.\n\n## Latest Trends in Artificial Intelligence\n\n### Machine Learning Advances\nOne of the most significant trends in AI is the continuous advancements in machine learning algorithms. Recent breakthroughs have propelled AI capabilities to new heights, enabling more accurate predictions, personalized recommendations, and enhanced decision-making processes across industries. From predictive analytics to image recognition, machine 

In [ ]:
from IPython.display import Markdown
Markdown(str(result))


# Unveiling the Future: Exploring the Latest Trends and Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has rapidly transformed from a futuristic concept to a ubiquitous presence in today's world, impacting various industries and reshaping the way we interact with technology. This blog post aims to delve into the latest trends, top players, and noteworthy news in the ever-evolving field of AI, offering insights into the advancements driving innovation and the key contributors shaping the industry landscape.

## Latest Trends in Artificial Intelligence

### Machine Learning Advances
One of the most significant trends in AI is the continuous advancements in machine learning algorithms. Recent breakthroughs have propelled AI capabilities to new heights, enabling more accurate predictions, personalized recommendations, and enhanced decision-making processes across industries. From predictive analytics to image recognition, machine learning is at the forefront of driving AI innovation.

### Natural Language Processing (NLP) Developments
Another key trend in AI revolves around Natural Language Processing (NLP) developments. The evolution of NLP technologies has revolutionized how machines understand and generate human language. Sentiment analysis, language translation, and conversational AI interfaces are just a few examples of how NLP is enhancing communication and user experiences in applications ranging from chatbots to virtual assistants.

### Robotics and Automation
AI's impact on robotics and automation is undeniable, with concepts like self-driving cars and drone technology pushing boundaries in autonomous systems. The integration of AI algorithms into robotics has not only increased efficiency and productivity but also opened new possibilities in sectors such as manufacturing, logistics, and healthcare. The future of work is being reimagined through AI-driven robotics and automation solutions.

## Key Players in the AI Industry

### Technology Giants
Tech titans like Google, Amazon, and Microsoft have played pivotal roles in shaping the AI industry through investments, acquisitions, and groundbreaking research. Their contributions to AI innovation span across various domains, from cloud computing services to AI-powered products and services, solidifying their positions as key players driving the AI revolution.

### Startups and Innovators
While established companies dominate the AI landscape, startups and innovators bring fresh perspectives and disruptive solutions to the table. Emerging AI startups are challenging traditional paradigms with niche offerings in areas such as healthcare, finance, and cybersecurity, showcasing the diversity and ingenuity present in the AI startup ecosystem.

### Academic and Research Institutions
Universities and research institutions serve as the breeding grounds for cutting-edge AI research and talent development. Their contributions to AI development through academic publications, collaborative projects, and fostering future AI leaders are instrumental in pushing the boundaries of innovation and addressing complex challenges in the field.

## Noteworthy News in Artificial Intelligence

### Ethical Considerations
As AI technologies continue to advance, ethical considerations surrounding AI development and deployment have come under intense scrutiny. The responsible use of AI, transparent algorithms, and data privacy are paramount in ensuring that AI benefits society while mitigating potential risks and biases inherent in AI systems.

### Industry Collaborations
Collaborations between AI companies and other industries have become increasingly common, fostering synergies and driving cross-sector innovation. Partnerships between AI startups and established corporations, as well as collaborations between AI researchers and industry practitioners, are leveraging collective expertise to solve real-world problems and propel AI applications to new heights.

### Regulatory Updates
The dynamic nature of the AI landscape means that regulatory frameworks must adapt to keep pace with technological advancements. Stay updated on the latest regulations and policies governing AI practices, as governments and regulatory bodies worldwide navigate the complexities of AI ethics, accountability, and compliance in an evolving digital era.

## Target Audience Analysis

This article caters to a diverse audience, including tech enthusiasts, industry professionals, and business leaders seeking to stay informed on the latest trends in AI. By addressing interests in cutting-edge technology, innovation, and industry disruptions, this content aims to provide valuable insights and actionable knowledge to those navigating the complex AI landscape.

## SEO Keywords
- Artificial Intelligence trends 2022
- Top AI companies
- Latest news on machine learning
- AI developments in robotics
- Responsible AI practices

## Resources
For in-depth insights into AI trends and developments, refer to industry reports from Gartner and Forrester Research, recent articles from reputable sources like MIT Technology Review and Forbes, and academic papers from leading institutions on AI ethics and regulations.

**Call to Action:** Stay ahead in the AI race by staying informed on the latest trends and developments. Subscribe to our newsletter for regular insights on Artificial Intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

B#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

In [23]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token=os.getenv("OPENAI_API_KEY"),
    task="text-generation",
)

C:\Users\PC\AppData\Local\Temp\ipykernel_18400\2724231878.py:3: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(
c:\Users\PC\anaconda3\envs\crewai-tutorial\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.getenv("OPENAI_API_KEY")

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).